### AMM Optimization, 1st model

In [4]:
using Gurobi, JuMP, CSV, Tables, Ipopt

In [10]:
# With an order amount of token x
function optimize_order_execution(liquidity, order_amount)

    amount_token_x_per_pool = liquidity[:, 1] 
    amount_token_y_per_pool = liquidity[:, 2]

    # Create a new model
    model = Model(Ipopt.Optimizer)

    # Create variables
    @variable(model, x[1:size(liquidity,1)] >= 0)

    # Set objective
    @NLobjective(model, Min, sum(amount_token_y_per_pool[i] / (amount_token_x_per_pool[i]+ x[i]) for i in 1:size(liquidity,1)))

    # Add constraint
    @constraint(model, sum(x) == order_amount)
    @constraint(model, x .<= amount_token_x_per_pool)

    # Optimize model
    optimize!(model)

    # Return the optimal solution
    return value.(x)
end

optimize_order_execution (generic function with 1 method)

## EXPERIMENT 1: AMM Optimization, no transaction costs

### Generating random data

We just need to preserve the current magnitude. Also AMMs need to be close to each other in the absence of transaction costs.

In [2]:
ETH_USDT = 1210;

In [9]:
ETH = rand(3000:3500, 12)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];

In [21]:
amount_token_x_per_pool = liquidity[:, 1]
amount_token_y_per_pool = liquidity[:, 2]

model = Model(Gurobi.Optimizer)

#@variable(model, 0 <= x[1:length(liquidity)] <= amount_token_x_per_pool[1:length(liquidity)])
size(liquidity,1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


12

In [74]:
split_trades = optimize_order_execution(test, 1000)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       12

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.4519955e+04 1.00e+03 1.39e-02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

12-element Vector{Float64}:
 114.81120175356594
 113.36517827613255
  40.14836805870186
 112.88300713600246
 190.83919846697503
  52.96541003058026
 142.14465525449506
  27.27983737100211
  26.783873794529306
  97.41065877431663
  81.3686092071725
   1.8765262275403233e-6